In [ ]:
from multiprocessing import Manager
from concurrent.futures import ThreadPoolExecutor
from threading import current_thread
from time import time
from urllib.request import urlopen
import requests
from lxml import html
from time import time

In [ ]:
"""
create a "thread safe" queue. thread safe
means that the queue (which is really a list)
can handle read/write from multiple thhreads.
"""
data_queue = Manager().Queue()

In [ ]:
def crawl(idx, my_queue):

    # create path
    bill_url = 'https://www.govinfo.gov/bulkdata/BILLSTATUS/115/hr/BILLSTATUS-115hr' + str(idx) + '.xml'
    bill_source = requests.get(bill_url)
    bill_ntree = html.document_fromstring(bill_source.content)

    # fetch result and put into queue
    result = bill_ntree.xpath('//recordedvote[descendant::chamber/text()="House"]/rollnumber/text()')
    my_queue.put(result)
    
    # track
    print(">> thread {} fetched document {}".format(current_thread().name, idx))

In [ ]:
n_threads = 10  # K threads means K x faster
n_documents = 20  # number of documents to crawl

start = time()

with ThreadPoolExecutor(max_workers=n_threads) as executor:
    for idx in range(1, n_documents):  
        _ = executor.submit(crawl, idx, data_queue)

print("execution time", time() - start)

In [ ]:
while not data_queue.empty():
    print(data_queue.get())